In [ ]:
import os
from openai import OpenAI
import rich
from google.colab import userdata

In [ ]:
api_key = userdata.get('OPENAI_API_KEY')
MODEL = "gpt-4o-mini"

openai = OpenAI(api_key=api_key)

Search the web for the latest information before generating a response

1) In Chat API, model always retrieves information from web before responding to your query

2) In Responses API, you enable web search by configuring it in the tools, and model can choose to serach the web or not based on the content of the input prompt.

To refine search results based on geography, you can specify the user's approximate location using parameters such as country, city, region, and/or timezone.

# Chat Completion API

https://platform.openai.com/docs/guides/tools-web-search?api-mode=chat

Always search the web first

Note: The `user_location` object can be configured as shown below to specify the approximate user location in the search options
```
web_search_options={
    "user_location": {
        "type": "approximate",
        "approximate": {
            "country": "GB",
            "city": "London",
            "region": "London",
        }
    },
},

```

In [ ]:
response = openai.chat.completions.create(
    model="gpt-4o-search-preview",
    web_search_options={
        "user_location": {
            "type": "approximate",
            "approximate": {
                "country": "PK",
                "city": "Karachi",
                "region": "Sindh",
            }
        },
    },
    messages=[
        {
            "role": "user",
            "content": "List the best restaurants for Seafood",
        }
    ],
)

print(response.choices[0].message.content)
# rich.print(response)
print(len(response.choices[0].message.annotations))
rich.print(response.choices[0].message.annotations)

Karachi, with its coastal location, boasts a vibrant seafood dining scene. Here are some of the top seafood restaurants in the city:

**[Kolachi Restaurant](https://www.google.com/maps/search/Kolachi+Restaurant%2C+Karachi%2C+Pakistan)**
_Karachi, Pakistan_
Situated along the Arabian Sea, Kolachi offers a picturesque dining experience with a diverse seafood menu, including their signature Kolachi Karahi—a flavorful blend of fresh fish, shrimp, and spices.

**[Okra](https://www.google.com/maps/search/Okra%2C+Karachi%2C+Pakistan)**
_Karachi, Pakistan_
Located in the Bukhari Commercial area, Okra specializes in Mediterranean and Asian-inspired seafood dishes, such as Grilled Sea Bass and Seafood Paella, all prepared with fresh ingredients.

**[The Patio](https://www.google.com/maps/search/The+Patio%2C+Karachi%2C+Pakistan)**
_Karachi, Pakistan_
Nestled in Frere Town, The Patio offers a cozy ambiance with outdoor seating and a menu featuring Mediterranean-inspired seafood like Grilled King P

[
    Annotation(
        type='url_citation',
        url_citation=AnnotationURLCitation(
            end_index=234,
            start_index=136,
            title='Kolachi Restaurant',
            url='https://www.google.com/maps/search/Kolachi+Restaurant%2C+Karachi%2C+Pakistan'
        )
    ),
    Annotation(
        type='url_citation',
        url_citation=AnnotationURLCitation(
            end_index=532,
            start_index=462,
            title='Okra',
            url='https://www.google.com/maps/search/Okra%2C+Karachi%2C+Pakistan'
        )
    ),
    Annotation(
        type='url_citation',
        url_citation=AnnotationURLCitation(
            end_index=829,
            start_index=749,
            title='The Patio',
            url='https://www.google.com/maps/search/The+Patio%2C+Karachi%2C+Pakistan'
        )
    ),
    Annotation(
        type='url_citation',
        url_citation=AnnotationURLCitation(
            end_index=1112,
            start_index=1029,
            title='Café Flo',
            url='https://www.google.com/maps/search/Caf%C3%A9+Flo%2C+Karachi%2C+Pakistan'
        )
    ),
    Annotation(
        type='url_citation',
        url_citation=AnnotationURLCitation(
            end_index=1409,
            start_index=1307,
            title='Savor Seafood Lounge',
            url='https://www.google.com/maps/search/Savor+Seafood+Lounge%2C+Karachi%2C+Pakistan'
        )
    ),
    Annotation(
        type='url_citation',
        url_citation=AnnotationURLCitation(
            end_index=1729,
            start_index=1593,
            title='N’eco’s Natural Store & Cafe',
            url='https://www.google.com/maps/search/N%E2%80%99eco%E2%80%99s+Natural+Store+%26+Cafe%2C+Karachi%2C+Pa
kistan'
        )
    ),
    Annotation(
        type='url_citation',
        url_citation=AnnotationURLCitation(
            end_index=2030,
            start_index=1948,
            title='Oyster Bay',
            url='https://www.google.com/maps/search/Oyster+Bay%2C+Karachi%2C+Pakistan'
        )
    ),
    Annotation(
        type='url_citation',
        url_citation=AnnotationURLCitation(
            end_index=2352,
            start_index=2242,
            title='Liaquat Bhai Fish Master',
            url='https://www.google.com/maps/search/Liaquat+Bhai+Fish+Master%2C+Karachi%2C+Pakistan'
        )
    ),
    Annotation(
        type='url_citation',
        url_citation=AnnotationURLCitation(
            end_index=2644,
            start_index=2554,
            title='Rashid Seafood',
            url='https://www.google.com/maps/search/Rashid+Seafood%2C+Karachi%2C+Pakistan'
        )
    ),
    Annotation(
        type='url_citation',
        url_citation=AnnotationURLCitation(
            end_index=2906,
            start_index=2812,
            title='Akram Fish Point',
            url='https://www.google.com/maps/search/Akram+Fish+Point%2C+Karachi%2C+Pakistan'
        )
    )
]

# Responses API

https://platform.openai.com/docs/guides/tools-web-search?api-mode=responses

Model will choose to search the web or not based on the content of the input prompt.

Note: The `user_location` object can be configured as shown below to specify the approximate user location in the search options

In the Responses API, you can see that the `user_location` object does not contain a nested `approximate` object, unlike in the Chat API.
```
"user_location": {
    "type": "approximate",
    "country": "PK",
    "city": "Lahore",
    "region": "Punjab",
}
```

Sometimes, the model may not feel the need to call the web search preview tool. In such cases, it might respond with a message asking the user to specify the location or return results that are not location-specific.

To see the difference try this example multiple times

In [ ]:
response = openai.responses.create(
    model=MODEL,
    tools=[{
        "type": "web_search_preview",
        "user_location": {
            "type": "approximate",
            "country": "PK",
            "city": "Lahore",
            "region": "Punjab",
        }
    }],
    # input="List the best restaurants for Seafood"
    input="List the best restaurants for Seafood from current location"
)

print(response.output_text)
rich.print(response.output)
rich.print(response.tools)
# rich.print(response)

Lahore offers a variety of excellent seafood restaurants that cater to diverse tastes. Here are some top recommendations:

**[Coastal Reef Seafood Restaurant](https://www.google.com/maps/search/Coastal+Reef+Seafood+Restaurant%2C+Lahore%2C+Pakistan)**
_Lahore, Pakistan_
A premier seafood spot in DHA, offering a wide variety of dishes in a cozy, family-friendly atmosphere.

**[The Fisherman's Wharf](https://www.google.com/maps/search/The+Fisherman%27s+Wharf%2C+Lahore%2C+Pakistan)**
_Lahore, Pakistan_
Inspired by coastal dining styles, this restaurant offers fresh seafood in a relaxed ambiance, located in the iconic Food Street area.

**[Siddique Fish Corner](https://www.google.com/maps/search/Siddique+Fish+Corner%2C+Lahore%2C+Pakistan)**
_Lahore, Pakistan_
Renowned for its fresh River Rahu, cooked with pure mustard oil to enhance flavor, offering a unique seafood experience.

**[Bashir Darul Mahi](https://www.google.com/maps/search/Bashir+Darul+Mahi%2C+Lahore%2C+Pakistan)**
_Lahore, Paki

[
    ResponseFunctionWebSearch(
        id='ws_67e38e65acc08192b6e1dde3137aed160fe795cc4c7994da',
        status='completed',
        type='web_search_call'
    ),
    ResponseOutputMessage(
        id='msg_67e38e683e848192b16db8f3081fd1db0fe795cc4c7994da',
        content=[
            ResponseOutputText(
                annotations=[
                    AnnotationURLCitation(
                        end_index=248,
                        start_index=125,
                        title='Coastal Reef Seafood Restaurant',
                        type='url_citation',
                        url='https://www.google.com/maps/search/Coastal+Reef+Seafood+Restaurant%2C+Lahore%2C+Pakist
an'
                    ),
                    AnnotationURLCitation(
                        end_index=482,
                        start_index=377,
                        title="The Fisherman's Wharf",
                        type='url_citation',
                        url='https://www.google.com/maps/search/The+Fisherman%27s+Wharf%2C+Lahore%2C+Pakistan'
                    ),
                    AnnotationURLCitation(
                        end_index=743,
                        start_index=642,
                        title='Siddique Fish Corner',
                        type='url_citation',
                        url='https://www.google.com/maps/search/Siddique+Fish+Corner%2C+Lahore%2C+Pakistan'
                    ),
                    AnnotationURLCitation(
                        end_index=984,
                        start_index=889,
                        title='Bashir Darul Mahi',
                        type='url_citation',
                        url='https://www.google.com/maps/search/Bashir+Darul+Mahi%2C+Lahore%2C+Pakistan'
                    ),
                    AnnotationURLCitation(
                        end_index=1211,
                        start_index=1128,
                        title='Sardar Fish',
                        type='url_citation',
                        url='https://www.google.com/maps/search/Sardar+Fish%2C+Lahore%2C+Pakistan'
                    ),
                    AnnotationURLCitation(
                        end_index=1434,
                        start_index=1351,
                        title='Caspian Sea',
                        type='url_citation',
                        url='https://www.google.com/maps/search/Caspian+Sea%2C+Lahore%2C+Pakistan'
                    ),
                    AnnotationURLCitation(
                        end_index=1659,
                        start_index=1558,
                        title='Blow Fish Restaurant',
                        type='url_citation',
                        url='https://www.google.com/maps/search/Blow+Fish+Restaurant%2C+Lahore%2C+Pakistan'
                    ),
                    AnnotationURLCitation(
                        end_index=1863,
                        start_index=1782,
                        title='Bistro 201',
                        type='url_citation',
                        url='https://www.google.com/maps/search/Bistro+201%2C+Lahore%2C+Pakistan'
                    )
                ],
                text="Lahore offers a variety of excellent seafood restaurants that cater to diverse tastes. Here 
are some top recommendations:\n\n**[Coastal Reef Seafood 
Restaurant](https://www.google.com/maps/search/Coastal+Reef+Seafood+Restaurant%2C+Lahore%2C+Pakistan)**\n_Lahore, 
Pakistan_\nA premier seafood spot in DHA, offering a wide variety of dishes in a cozy, family-friendly 
atmosphere.\n\n**[The Fisherman's 
Wharf](https://www.google.com/maps/search/The+Fisherman%27s+Wharf%2C+Lahore%2C+Pakistan)**\n_Lahore, 
Pakistan_\nInspired by coastal dining styles, this restaurant offers fresh seafood in a relaxed ambiance, located 
in the iconic Food Street area.\n\n**[Siddique Fish 
Corner](https://www.google.com/maps/search/Siddique+Fish+Corner%2C+Lahore%2C+Pakistan)**\n_Lahore, 
Pakistan_\nRenowned for its fresh River Rahu, coo

[
    WebSearchTool(
        type='web_search_preview',
        search_context_size='medium',
        user_location=UserLocation(
            type='approximate',
            city='Lahore',
            country='PK',
            region='Punjab',
            timezone=None
        )
    )
]